In [115]:
import pandas as pd
import numpy as np
import string
from sklearn.model_selection import train_test_split
from pandas.api.types import CategoricalDtype 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [116]:
train_raw = pd.read_csv('/kaggle/input/cat-in-the-dat/train.csv')
test = pd.read_csv('/kaggle/input/cat-in-the-dat/test.csv')

In [117]:
train_y = train_raw['target']
train_raw = train_raw.drop(['target'], axis = 1)
train = train_raw.append(test)

In [118]:
print("column: train unique values - test unique values")
for column in train.columns:
    print(column + ": " + str(len(train[:300000][column].unique())) + " --> " + str(len(train[300000:][column].unique())))

column: train unique values - test unique values
id: 300000 --> 200000
bin_0: 2 --> 2
bin_1: 2 --> 2
bin_2: 2 --> 2
bin_3: 2 --> 2
bin_4: 2 --> 2
nom_0: 3 --> 3
nom_1: 6 --> 6
nom_2: 6 --> 6
nom_3: 6 --> 6
nom_4: 4 --> 4
nom_5: 222 --> 222
nom_6: 522 --> 522
nom_7: 1220 --> 1219
nom_8: 2215 --> 2214
nom_9: 11981 --> 11839
ord_0: 3 --> 3
ord_1: 5 --> 5
ord_2: 6 --> 6
ord_3: 15 --> 15
ord_4: 26 --> 26
ord_5: 192 --> 192
day: 7 --> 7
month: 12 --> 12


In [119]:

mapping_bins34 = {'T' : 1, 'F' : 0, 'Y' : 1, 'N' : 0}

train['bin_3'] = train['bin_3'].map(mapping_bins34)
train['bin_4'] = train['bin_4'].map(mapping_bins34)
     

In [120]:
for col in ['ord_1','ord_2']:
    print(col)
    print(train[col].unique())
     

ord_1
['Grandmaster' 'Expert' 'Novice' 'Contributor' 'Master']
ord_2
['Cold' 'Hot' 'Lava Hot' 'Boiling Hot' 'Freezing' 'Warm']


In [121]:
ord1_dic = {'Novice' : 0, 'Contributor' : 1, 'Expert' : 2, 'Master' : 3, 'Grandmaster' : 4}
ord2_dic = {'Freezing' : 0, 'Cold' : 1, 'Warm' : 2, 'Hot' : 3, 'Boiling Hot' : 4, 'Lava Hot' : 5}

train['ord_1'] = train['ord_1'].map(ord1_dic)
train['ord_2'] = train['ord_2'].map(ord2_dic)

In [122]:
#sort ord_3 alphabetically then insert into dic then map new values
ord3 = sorted(train['ord_3'].unique())
ord3_dic = {}

i=1
for letter in ord3:
    ord3_dic[letter] = i
    i += 1
    
train['ord_3'] = train['ord_3'].map(ord3_dic)
     

In [123]:
#sort ord_4 alphabetically then insert into dic then map new values
ord4 = sorted(train['ord_4'].unique())
ord4_dic = {}

i=1
for letter in ord4:
    ord4_dic[letter] = i
    i += 1
    
train['ord_4'] = train['ord_4'].map(ord4_dic)

In [124]:
train['ord_5_oe_add'] = train['ord_5'].apply(lambda x:sum([(string.ascii_letters.find(letter)+1) for letter in x]))
train = train.drop(['ord_5'], axis = 1)
     

In [125]:
#One hot encode nominal features
train = pd.get_dummies(train, columns=['nom_0', 'nom_1', 'nom_2', 'nom_3', 'nom_4'],\
                          prefix=['nom_0', 'nom_1', 'nom_2', 'nom_3', 'nom_4'], drop_first=True)

In [126]:
high_card_nom = ['nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9']

for col in high_card_nom:
    train[f'hash_{col}'] = train[col].apply( lambda x: hash(str(x)) % 5000 )
    train = train.drop([col], axis = 1)
     

In [127]:
train_final = train[:len(train_raw)]
test_final = train[(len(train_raw)):]

In [128]:
X_train,X_test,y_train,y_test = train_test_split(train_final,train_y,test_size=0.2)
     

In [129]:
# from sklearn.model_selection import GridSearchCV
# grid={"C":np.logspace(-3,3,7), "penalty":["l1","l2"],'solver':"lbfgs"} # l1 lasso l2 ridge
# clf=LogisticRegression()
# clf_cv=GridSearchCV(clf,grid,cv=10)
# clf_cv.fit(X_train,y_train)

In [130]:
# print("tuned hpyerparameters :(best parameters) ",clf_cv.best_params_)
# print("accuracy :",clf_cv.best_score_)

In [131]:
clf = LogisticRegression(C=0.1, solver="lbfgs", max_iter=5000, n_jobs=-1)

In [132]:
clf.fit(X_train, y_train)

LogisticRegression(C=0.1, max_iter=5000, n_jobs=-1)

In [133]:
y_pred = clf.predict_proba(X_test)[:, 1]

In [134]:
final_pred = clf.predict_proba(test_final)[:, 1]

In [135]:
pd.DataFrame({"id": test['id'], "target": final_pred}).to_csv("submission.csv", index=False)